In [1]:
import pandas as pd

In [2]:
directory = '/media/share/DataSaves/wikitable_extract_220814_test'


In [3]:
import glob
test = glob.glob(directory + '/*.parquet')

In [4]:
# !pip install py-wikimarkup

In [48]:
from wikimarkup.parser import Parser
from bs4 import BeautifulSoup

parser = Parser()

def get_tables(markdowntext):
    output = []
    try:
        html = parser.parse(markdowntext)
        soup = BeautifulSoup(html, 'html.parser')
        tables = soup.find_all('table')
        df_tables = pd.read_html(html)
        if len(tables) != len(df_tables):
            print("tables offset..")
            return []
        output = []
        for i, table in enumerate(tables):
            # all previous headings in order
            headings = []
            for k in range(6):
                h = table.find_previous_sibling(f'h{k}')
                headings.append(
                    h.text if h else ''
                )
            output.append((i, "\n".join(headings), df_tables[i]))
    except Exception as e:
        print("Failed to parse...")
    return output

def get_all_tables(path):
    data = pd.read_parquet(path)
    data = data['page_text'].groupby(data['page_title_text']).first()
    for title, text in data.iteritems():
        result = get_tables(text)
        for row in result:
            yield title, *row

In [6]:
import time
st = time.time()
results = []
for i, x in enumerate(get_all_tables(test[125])):
    results.append(x)
    if i%10 == 0:
        print(i, time.time() - st)
    if i > 500:
        break

0 2.017167568206787
10 2.5265681743621826
20 2.6810081005096436
30 2.862239360809326
40 3.198089122772217
50 3.198174238204956
60 3.253628730773926
70 3.525266170501709
80 3.66201114654541
90 3.9148833751678467
100 3.91496205329895
110 3.9149725437164307
120 4.185628175735474
130 4.340177059173584
140 4.754835605621338
150 4.963443279266357
160 5.167338609695435
tables offset..
170 5.66095232963562
180 5.8487021923065186
190 5.945636987686157
200 6.134185552597046
210 6.682735919952393
220 7.268476247787476
230 7.783344268798828
240 8.288628578186035
250 8.63939356803894
260 8.929021835327148
270 9.133349180221558
280 9.415085792541504
290 9.566310405731201
300 9.833694219589233
310 9.833808660507202
320 10.13634443283081
330 10.327559232711792
340 10.607861042022705
350 10.777730703353882
360 10.803031921386719
370 11.178155660629272
380 11.302590370178223
390 11.385343313217163
400 11.72278881072998
410 11.862169742584229
420 11.972707271575928
430 12.320104598999023
440 12.551195144

In [16]:
import random
page, index, text, frame = random.choice(results)

In [45]:
results[112]

('1923 AAA Championship Car season',
 1,
 ',,Leading National Championship standings,,,',
    #                                 Driver  \
 0  1                       [[Eddie Hearne]]   
 1  2  [[James Anthony Murphy|Jimmy Murphy]]   
 2  3                       [[Bennett Hill]]   
 3  4                        [[Harry Hartz]]   
 4  5                       [[Tommy Milton]]   
 
                                                  Car  Points  
 0              [[Harry Miller (auto racing)|Miller]]    1882  
 1              [[Harry Miller (auto racing)|Miller]]    1350  
 2  [[Harry Miller (auto racing)|Miller]], [[Duese...     955  
 3  [[Duesenberg]], [[Harry Miller (auto racing)|M...     820  
 4              [[Harry Miller (auto racing)|Miller]]     810  )

In [17]:
url = f"https://en.wikipedia.org/wiki/{page.replace(' ', '_')}"

In [24]:
print(url)

https://en.wikipedia.org/wiki/1952_South_Korean_presidential_election


In [37]:
dfs = pd.read_html(url)[7]  # note, 7 may be wrong, please consider changing to other numbers

In [38]:
dfs.head()

Region       Syngman Rhee                           Cho Bong-am  \
              Region Unnamed: 1_level_1 Unnamed: 2_level_1 Unnamed: 3_level_1   
              Region              Votes                  %              Votes   
0              Seoul             205300               82.2              25631   
1           Gyeonggi             657174               87.7              44967   
2            Gangwon             366583               92.4              10516   
3  North Chungcheong             386665               86.7              25875   
4  South Chungcheong             636061               82.4              56590   

                            Yi Si-yeong                          Shin Heung-u  \
  Unnamed: 4_level_1 Unnamed: 5_level_1 Unnamed: 6_level_1 Unnamed: 7_level_1   
                   %              Votes                  %              Votes   
0               10.3              14883                6.0               3923   
1                6.0              34704                4.6              12891   
2                2.7              13378                3.4               6305   
3                5.8              23006                5.2              10409   
4                7.3              58754                7.6              20947   

                      
  Unnamed: 8_level_1  
                   %  
0                1.6  
1                1.7  
2                1.6  
3                2.3  
4                2.7

In [ ]:
'1963 Campeonato Brasileiro Série A'.replace(' ', '_')

In [ ]:
test[0]

In [ ]:
trial.columns

In [ ]:
# print(trial['page_title_text'][15])
print(trial['template_internals'][15])

In [ ]:
# |- seems to be a row break
# | {...} |seems to be a cell break

